### Cython Optimizations

In this notebook, we explore improvements using Numba and Cython.

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

### Baseline

In [339]:
# Baseline code
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days):
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    ax.figure.savefig('figures/sir_plot.png')
    plt.close()

def run_simulation(beta, gamma, public_trans, thresh, days, OD_path):
    OD = np.genfromtxt(OD_path, delimiter=',')
    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh=200
    first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]
    
    R0 = beta/gamma
    beta_vec = np.random.gamma(1.6, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = np.full(locs_len, public_trans)

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = []
    susceptible_pop_norm = []
    recovered_pop_norm = []

    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = OD*infected_mat
        OD_infected = np.round(OD_infected)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec)
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm.append(I)
        susceptible_pop_norm.append(S)
        recovered_pop_norm.append(R)
    
    plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm



In [340]:
%timeit -n 3 run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

100%|██████████| 100/100 [00:06<00:00, 15.80it/s]


8.64 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


The baseline code with minor restructuring has an average run time is 8.64s.

In [273]:
# Figure out which part of the code is taking the longest using lprun
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [338]:
%lprun -f run_simulation (I, S, R) = run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

100%|██████████| 100/100 [00:05<00:00, 17.31it/s]


Timer unit: 1e-06 s

Total time: 9.7423 s
File: <ipython-input-337-84860c97039b>
Function: run_simulation at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    22                                           def run_simulation(beta, gamma, public_trans, thresh, days, OD_path):
    23         1    3818488.0 3818488.0     39.2      OD = np.genfromtxt(OD_path, delimiter=',')
    24                                               # initialize the population vector from the origin-destination flow matrix
    25         1       4288.0   4288.0      0.0      N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    26         1          2.0      2.0      0.0      locs_len = len(N_k)                 # number of locations
    27         1          9.0      9.0      0.0      SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    28         1          6.0      6.0      0.0      SIR[:,0] = N_k                      # initialize the S group with the respective populations
    29         1          1.0      1.0      0.0      thresh=200
    30         1         91.0     91.0      0.0      first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    31                                               # NOTE: this is arbitrary but not actually random.... 
    32         1          9.0      9.0      0.0      SIR[:, 0] = SIR[:, 0] - first_infections
    33         1          7.0      7.0      0.0      SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group
    34                                           
    35                                               # row normalize the SIR matrix for keeping track of group proportions
    36         1         41.0     41.0      0.0      row_sums = SIR.sum(axis=1)
    37         1         20.0     20.0      0.0      SIR_n = SIR / row_sums[:, np.newaxis]
    38                                               
    39         1          1.0      1.0      0.0      R0 = beta/gamma
    40         1        137.0    137.0      0.0      beta_vec = np.random.gamma(1.6, 2, locs_len)
    41         1         17.0     17.0      0.0      gamma_vec = np.full(locs_len, gamma)
    42         1          8.0      8.0      0.0      public_trans_vec = np.full(locs_len, public_trans)
    43                                           
    44                                               # make copy of the SIR matrices 
    45         1          5.0      5.0      0.0      SIR_sim = SIR.copy()
    46         1          5.0      5.0      0.0      SIR_nsim = SIR_n.copy()
    47                                           
    48                                               # run model
    49                                               #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    50         1          1.0      1.0      0.0      infected_pop_norm = []
    51         1          1.0      1.0      0.0      susceptible_pop_norm = []
    52         1          1.0      1.0      0.0      recovered_pop_norm = []
    53                                           
    54       101      49135.0    486.5      0.5      for time_step in tqdm(range(days)):
    55       100    1010624.0  10106.2     10.4          infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
    56       100    3254668.0  32546.7     33.4          OD_infected = OD*infected_mat
    57       100    1012173.0  10121.7     10.4          OD_infected = np.round(OD_infected)
    58       100     238400.0   2384.0      2.4          inflow_infected = OD_infected.sum(axis=0)
    59       100       3263.0     32.6      0.0          inflow_infected = np.round(inflow_infected*public_trans_vec)
    60                                                   #print('total infected inflow: ', inflow_infected.sum())
    61       100     238517.0   2385.2      2.4          new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
    62       100       1028.0     10.3      0.0          new_recovered = gamma_vec*SIR_sim[:, 1]
    63       100       2066.0     20.7      0.0          new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
    64       100       1027.0     10.3      0.0          SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
    65       100       1007.0     10.1      0.0          SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
    66       100        716.0      7.2      0.0          SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
    67       100       2414.0     24.1      0.0          SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
    68                                                   # recompute the normalized SIR matrix
    69       100       4324.0     43.2      0.0          row_sums = SIR_sim.sum(axis=1)
    70       100       2144.0     21.4      0.0          SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
    71       100       1662.0     16.6      0.0          S = SIR_sim[:,0].sum()/N_k.sum()
    72       100       1037.0     10.4      0.0          I = SIR_sim[:,1].sum()/N_k.sum()
    73       100       1003.0     10.0      0.0          R = SIR_sim[:,2].sum()/N_k.sum()
    74                                                   #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
    75                                                   #print('\n')
    76       100        237.0      2.4      0.0          infected_pop_norm.append(I)
    77       100         89.0      0.9      0.0          susceptible_pop_norm.append(S)
    78       100        114.0      1.1      0.0          recovered_pop_norm.append(R)
    79                                               
    80         1      93513.0  93513.0      1.0      plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    81         1          2.0      2.0      0.0      return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm

### Line Profiler Analysis

Using lprun, we see that most of the time is spent loading in the Origin-Destination flow matrix (accounts for 39.2% of the time) and computing the element-wise matrix multiplication OD\*infected_mat and rounding it (accounts for 43.8% of the time). Note that the matrix multiplication occurs in a for-loop that runs multiple times while the loading of the matrix occurs only once. The percentage of time needed to perform all of the matrix multiplications at all of the time steps is roughly the same as the time needed to load in the matrix. We will first focus on trying read the OD matrix faster.

### Numpy matrix loading

In [334]:
%%timeit -n3
OD = np.genfromtxt('data/Yerevan_OD_coronavirus.csv', delimiter=',')

2.21 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [335]:
%%timeit -n3
OD = np.genfromtxt('data/Yerevan_OD_coronavirus.csv', delimiter=',', dtype=np.float64)

2.1 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


Using numpy to do the matrix loading, we see that it takes $\approx 2$s to load in the matrix, even when we tell numpy what type of data the matrix will have.

### Loading with Pandas

In [280]:
import pandas as pd 

In [336]:
%%timeit -n3
OD = pd.read_csv('data/Yerevan_OD_coronavirus.csv', header=None, dtype=np.float64).to_numpy()

398 ms ± 6.68 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


Using the pandas library to read in the csv file and then converting it to a numpy array proves to be significantly faster ($\approx 7$x).

### Improving Matrix Multiplication - Cython

In [302]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [325]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days):
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    ax.figure.savefig('figures/sir_plot.png')
    plt.close()

def run_simulation(beta, gamma, public_trans, thresh, days, OD_path):
    OD = np.genfromtxt(OD_path, delimiter=',')
    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh=200
    first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]
    
    R0 = beta/gamma
    beta_vec = np.random.gamma(1.6, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = np.full(locs_len, public_trans)

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = []
    susceptible_pop_norm = []
    recovered_pop_norm = []

    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = OD*infected_mat
        OD_infected = np.round(OD_infected)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec)
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm.append(I)
        susceptible_pop_norm.append(S)
        recovered_pop_norm.append(R)
    
    plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm

In [326]:
%timeit -n 3 run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

100%|██████████| 100/100 [00:06<00:00, 15.63it/s]


8.19 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


We see a slight speed up when compiling with Cython without specifying the types of the variables.

### Cython - Specifying Types

In [323]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
cimport numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt

def plot_SIR_curve(np.ndarray[np.float64_t, ndim=1] infected_pop_norm, np.ndarray[np.float64_t, ndim=1] susceptible_pop_norm, np.ndarray[np.float64_t, ndim=1] recovered_pop_norm, int days):
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    ax.figure.savefig('figures/sir_plot.png')
    plt.close()

def run_simulation(np.float64_t beta, np.float64_t gamma, np.float64_t public_trans, int thresh, int days, str OD_path):
    cdef np.ndarray[np.float64_t, ndim=2] OD, SIR, SIR_n, SIR_sim, SIR_nsim, infected_mat, OD_infected 
    cdef np.ndarray[np.float64_t, ndim=1] N_k, first_infections, row_sums, beta_vec, gamma_vec, public_trans_vec, inflow_infected, new_infect, new_recovered
    cdef int locs_len, time_step
    cdef np.float64_t R0, S, I, R
    cdef np.ndarray[np.float64_t, ndim=1] infected_pop_norm, susceptible_pop_norm, recovered_pop_norm 
    
    OD = np.genfromtxt(OD_path, delimiter=',')
    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh=200
    first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]
    
    R0 = beta/gamma
    beta_vec = np.random.gamma(1.6, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = np.full(locs_len, public_trans)

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = np.zeros(days)
    susceptible_pop_norm = np.zeros(days)
    recovered_pop_norm = np.zeros(days)

    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = OD*infected_mat
        OD_infected = np.round(OD_infected)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec)
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm[time_step] = I 
        susceptible_pop_norm[time_step] = S 
        recovered_pop_norm[time_step] = R
    
    plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm

In [324]:
%timeit -n 3 run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

100%|██████████| 100/100 [00:06<00:00, 16.26it/s]


8.18 s ± 78.8 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


Negligible difference in performance between basic cython compilation and specifying types. 

### Cython + Faster Matrix Loading + Removing tqdm

In [3]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import pandas as pd 
import numpy as np
cimport numpy as np 
import matplotlib.pyplot as plt

def plot_SIR_curve(np.ndarray[np.float64_t, ndim=1] infected_pop_norm, np.ndarray[np.float64_t, ndim=1] susceptible_pop_norm, np.ndarray[np.float64_t, ndim=1] recovered_pop_norm, int days):
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    ax.figure.savefig('figures/sir_plot.png')
    plt.close()

def run_simulation(np.float64_t beta, np.float64_t gamma, np.float64_t public_trans, int thresh, int days, str OD_path):
    cdef np.ndarray[np.float64_t, ndim=2] OD, SIR, SIR_n, SIR_sim, SIR_nsim, infected_mat, OD_infected 
    cdef np.ndarray[np.float64_t, ndim=1] N_k, first_infections, row_sums, beta_vec, gamma_vec, public_trans_vec, inflow_infected, new_infect, new_recovered
    cdef int locs_len, time_step
    cdef np.float64_t R0, S, I, R
    cdef np.ndarray[np.float64_t, ndim=1] infected_pop_norm, susceptible_pop_norm, recovered_pop_norm 
    
    OD = pd.read_csv(OD_path, header=None, dtype=np.float64).to_numpy()
    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh=200
    first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]
    
    R0 = beta/gamma
    beta_vec = np.random.gamma(1.6, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = np.full(locs_len, public_trans)

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = np.zeros(days)
    susceptible_pop_norm = np.zeros(days)
    recovered_pop_norm = np.zeros(days)

    for time_step in range(days):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = OD*infected_mat
        OD_infected = np.round(OD_infected)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec)
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm[time_step] = I 
        susceptible_pop_norm[time_step] = S 
        recovered_pop_norm[time_step] = R
    
    plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm

UsageError: Cell magic `%%cython` not found.


In [328]:
%timeit -n 3 run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

3.71 s ± 67 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [329]:
%lprun -f run_simulation (I, S, R) = run_simulation(1.6, 0.04, 0.5, 200, 100, 'data/Yerevan_OD_coronavirus.csv')

Timer unit: 1e-06 s

Total time: 3.83334 s
File: /Users/benstadnick/.ipython/cython/_cython_magic_1959eba6153881ae8278eed514667ee6.pyx
Function: run_simulation at line 28

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    28                                           def run_simulation(np.float64_t beta, np.float64_t gamma, np.float64_t public_trans, int thresh, int days, str OD_path):
    29                                               cdef np.ndarray[np.float64_t, ndim=2] OD, SIR, SIR_n, SIR_sim, SIR_nsim, infected_mat, OD_infected 
    30                                               cdef np.ndarray[np.float64_t, ndim=1] N_k, first_infections, row_sums, beta_vec, gamma_vec, public_trans_vec, inflow_infected, new_infect, new_recovered
    31                                               cdef int locs_len, time_step
    32                                               cdef np.float64_t R0, S, I, R
    33                                               cdef np.ndarray[np.float64_t, ndim=1] infected_pop_norm, susceptible_pop_norm, recovered_pop_norm 
    34                                               
    35         1     531322.0 531322.0     13.9      OD = pd.read_csv(OD_path, header=None, dtype=np.float64).to_numpy()
    36                                               # initialize the population vector from the origin-destination flow matrix
    37         1       3796.0   3796.0      0.1      N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    38         1          1.0      1.0      0.0      locs_len = len(N_k)                 # number of locations
    39         1          6.0      6.0      0.0      SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    40         1          4.0      4.0      0.0      SIR[:,0] = N_k                      # initialize the S group with the respective populations
    41         1          0.0      0.0      0.0      thresh=200
    42         1         75.0     75.0      0.0      first_infections = np.where(SIR[:, 0]<=thresh, SIR[:, 0]//20, 0)   # for demo purposes, randomly introduce infections
    43                                               # NOTE: this is arbitrary but not actually random.... 
    44         1          7.0      7.0      0.0      SIR[:, 0] = SIR[:, 0] - first_infections
    45         1          6.0      6.0      0.0      SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group
    46                                           
    47                                               # row normalize the SIR matrix for keeping track of group proportions
    48         1         35.0     35.0      0.0      row_sums = SIR.sum(axis=1)
    49         1         19.0     19.0      0.0      SIR_n = SIR / row_sums[:, np.newaxis]
    50                                               
    51         1          0.0      0.0      0.0      R0 = beta/gamma
    52         1        115.0    115.0      0.0      beta_vec = np.random.gamma(1.6, 2, locs_len)
    53         1         17.0     17.0      0.0      gamma_vec = np.full(locs_len, gamma)
    54         1          8.0      8.0      0.0      public_trans_vec = np.full(locs_len, public_trans)
    55                                           
    56                                               # make copy of the SIR matrices 
    57         1          6.0      6.0      0.0      SIR_sim = SIR.copy()
    58         1          5.0      5.0      0.0      SIR_nsim = SIR_n.copy()
    59                                           
    60                                               # run model
    61                                               #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    62         1          3.0      3.0      0.0      infected_pop_norm = np.zeros(days)
    63         1          3.0      3.0      0.0      susceptible_pop_norm = np.zeros(days)
    64         1          2.0      2.0      0.0      recovered_pop_norm = np.zeros(days)
    65                                           
    66         1          0.0      0.0      0.0      for time_step in range(days):
    67       100     877018.0   8770.2     22.9          infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
    68       100    1033410.0  10334.1     27.0          OD_infected = OD*infected_mat
    69       100     913327.0   9133.3     23.8          OD_infected = np.round(OD_infected)
    70       100     180775.0   1807.8      4.7          inflow_infected = OD_infected.sum(axis=0)
    71       100       2869.0     28.7      0.1          inflow_infected = np.round(inflow_infected*public_trans_vec)
    72                                                   #print('total infected inflow: ', inflow_infected.sum())
    73       100     177611.0   1776.1      4.6          new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
    74       100       1073.0     10.7      0.0          new_recovered = gamma_vec*SIR_sim[:, 1]
    75       100       1873.0     18.7      0.0          new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
    76       100        854.0      8.5      0.0          SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
    77       100        827.0      8.3      0.0          SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
    78       100        598.0      6.0      0.0          SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
    79       100       2265.0     22.6      0.1          SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
    80                                                   # recompute the normalized SIR matrix
    81       100       3985.0     39.9      0.1          row_sums = SIR_sim.sum(axis=1)
    82       100       2049.0     20.5      0.1          SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
    83       100       1483.0     14.8      0.0          S = SIR_sim[:,0].sum()/N_k.sum()
    84       100        943.0      9.4      0.0          I = SIR_sim[:,1].sum()/N_k.sum()
    85       100        881.0      8.8      0.0          R = SIR_sim[:,2].sum()/N_k.sum()
    86                                                   #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
    87                                                   #print('\n')
    88       100         57.0      0.6      0.0          infected_pop_norm[time_step] = I 
    89       100         57.0      0.6      0.0          susceptible_pop_norm[time_step] = S 
    90       100         53.0      0.5      0.0          recovered_pop_norm[time_step] = R
    91                                               
    92         1      87890.0  87890.0      2.3      plot_SIR_curve(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, days)
    93         1       8016.0   8016.0      0.2      return infected_pop_norm, susceptible_pop_norm, recovered_pop_norm

By using Cython, changing how the OD matrix is loaded into the program, and removing the unnecessary tqdm functionality to show progress, we half the time it takes to run a simulation ($\approx 3.7$s). Examining the line profiling results, we see that time spent loading in the matrix is reduced (now $\approx 13.9\%$). Also notice that the percentages of time spent between rounding OD_infected and computing OD_infected are closer together, which suggests that we were able to speed up the elmentwise multiplication using Cython.